<a href="https://colab.research.google.com/github/doomsday4/Heat-Transfer-in-Advanced-Manufacturing-using-PINN/blob/main/model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import sys

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
# import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import interpolate
from scipy.interpolate import interp2d

import tensorflow as tf
import numpy as np

class SolidificationPINN:
    def __init__(self, x0, tem0, tb, X_f, X_tem, layers, lb, ub):
        self.lb = lb
        self.ub = ub

        self.x0 = x0
        self.tem0 = tem0
        self.tb = tb
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]
        self.tem_f = X_tem

        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)

        self.x0_tf = tf.convert_to_tensor(x0, dtype=tf.float64)
        self.tem0_tf = tf.convert_to_tensor(tem0, dtype=tf.float64)
        self.tb_tf = tf.convert_to_tensor(tb, dtype=tf.float64)
        self.x_f_tf = tf.convert_to_tensor(self.x_f, dtype=tf.float64)
        self.t_f_tf = tf.convert_to_tensor(self.t_f, dtype=tf.float64)
        self.tem_f_tf = tf.convert_to_tensor(self.tem_f, dtype=tf.float64)

        self.sess = tf.compat.v1.Session()

        self.learning_rate = tf.compat.v1.placeholder(tf.float64, shape=[])
        self.tem0_pred = self.net_uv(self.x0_tf, tf.zeros_like(self.x0_tf))
        self.f_u_pred = self.net_f_uv(self.x_f_tf, self.t_f_tf)

        self.loss = tf.reduce_mean(tf.square(self.tem0_tf - self.tem0_pred)) + \
                    tf.reduce_mean(tf.square(self.f_u_pred))

        self.optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate)
        self.train_op = self.optimizer.minimize(self.loss)

        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        for l in range(len(layers) - 1):
            W = self.xavier_init(size=[layers[l], layers[l + 1]])
            b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64), dtype=tf.float64)

    def forward_pass(self, H):
        for l in range(len(self.layers) - 2):
            W = self.weights[l]
            b = self.biases[l]
            H = tf.nn.swish(tf.add(tf.matmul(H, W), b))
        W = self.weights[-1]
        b = self.biases[-1]
        H = tf.add(tf.matmul(H, W), b)
        return H

    def net_uv(self, x, t):
        X = tf.concat([x, t], axis=1)
        uv = self.forward_pass(X)
        return uv

    def net_f_uv(self, x, t):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            uv = self.net_uv(x, t)
            uv_x = tape.gradient(uv, x)
        uv_t = tape.gradient(uv, t)
        uv_xx = tape.gradient(uv_x, x)
        del tape
        f_uv = uv_t - uv_xx
        return f_uv

    def train(self, epochs, learning_rate):
        for epoch in range(epochs):
            self.sess.run(self.train_op, feed_dict={self.learning_rate: learning_rate})
            if epoch % 100 == 0:
                loss_value = self.sess.run(self.loss, feed_dict={self.learning_rate: learning_rate})
                print(f'Epoch: {epoch}, Loss: {loss_value}')

    def predict(self, X_star):
        X_star = tf.convert_to_tensor(X_star, dtype=tf.float64)
        tem_star = self.sess.run(self.net_uv(X_star[:, 0:1], X_star[:, 1:2]))
        ftem_star = self.sess.run(self.net_f_uv(X_star[:, 0:1], X_star[:, 1:2]))
        return tem_star, ftem_star

import numpy as np
import scipy.io
from scipy.interpolate import interp2d
import time
from pyDOE import lhs
import tensorflow as tf

if __name__ == "__main__":
    noise = 0.0

    ltem = 298.15
    utem = 973.15
    eps = 0.02

    # Domain bounds
    lb = np.array([-0.4, 5.0])
    ub = np.array([0.4, 10.0])

    N0 = 300
    N_b = 100
    N_f = 10000
    num_hidden = 8
    layers = [2] + num_hidden * [200] + [1]

    data = scipy.io.loadmat('thermal_fine.mat')
    x = data['x'].flatten()[:, None]
    t = data['tt'].flatten()[:, None]
    Exact = data['Tem']
    Exact_tem = np.real(Exact)

    ftem = interp2d(x, t, Exact.T)

    X, T = np.meshgrid(x, t)
    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

    idx_x = np.random.choice(x.shape[0], N0, replace=False)
    x0 = x[idx_x, :]
    tem0 = Exact_tem[idx_x, 0:1]

    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t, :]

    X_f = lb + (ub - lb) * lhs(2, N_f)
    X_f = X_f[np.argsort(X_f[:, 0])]
    X_tem = ftem(X_f[:, 0], 0).flatten()[:, None]

    model = SolidificationPINN(x0, tem0, tb, X_f, X_tem, layers, lb, ub)

    start_time = time.time()
    model.train(epochs=1000, learning_rate=0.01)
    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))

    tem_pred, ftem_pred = model.predict(X_star)
    np.savetxt('predict_xT.txt', X_star)
    np.savetxt('predict_tem.txt', tem_pred)
    np.savetxt('predict_ftem.txt', ftem_pred)

<ipython-input-8-6374083f4725>:144: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  ftem = interp2d(x, t, Exact.T)
<ipython-input-8-6374083f4725>:158: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grids use `RegularGri

Epoch: 0, Loss: 513610.9637938071
Epoch: 100, Loss: 2557.692966414559
Epoch: 200, Loss: 47.284276287188106
Epoch: 300, Loss: 82.50186519952989
Epoch: 400, Loss: 394.02291045777


KeyboardInterrupt: 